   Ce reseau de neurones estime le meilleur valeur de dt à partir de dx et alpha pour un probléme de diffusion 1D. Il reçoit un tableu avec des valeurs de alpha, dx, dt, stabilité, nombre de interations, erreur vers la solution analytique et le     
   
   tmax <<<<<<<<<<<<<<<corrigir.

Le code filtre les donnes et crée un tableau composé de alpha, dx et dt avec seulment les cas stables et pour chaque paire alpha-dx il prend seulment le cas avec le plus petit valeur maximale da la variable.

In [2]:
pip install tensorflow

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import sys
import datetime, os
from sklearn import metrics
from sklearn import preprocessing


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

2.3.1


In [25]:
df = pd.DataFrame(np.array(([0.5, 0.5, 0.05, 1, 100, 1, 5],
[0.5, 0.5, 0.5,  0, 10,  1, 5],
[0.5, 2,   0.05, 1, 100, 1, 5],
[0.5, 2,   0.5,  1, 10,  1, 5],
[1,   0.5, 0.05, 1, 100, 1, 5],
[1,   0.5, 0.5,  0, 10,  1, 5],
[1,   2,   0.05, 1, 100, 1, 5],
[1,   2,   0.5,  1, 10,  1, 5],
[1.5, 0.5, 0.5,  0, 10,  1, 5],
[1.5, 2,   0.5,  1, 10,  1, 5]
)))
df

,0,1,2,3,4,5,6
0,0.5,0.5,0.05,1.0,100.0,1.0,5.0
1,0.5,0.5,0.50,0.0,10.0,1.0,5.0
2,0.5,2.0,0.05,1.0,100.0,1.0,5.0
3,0.5,2.0,0.50,1.0,10.0,1.0,5.0
4,1.0,0.5,0.05,1.0,100.0,1.0,5.0
5,1.0,0.5,0.50,0.0,10.0,1.0,5.0
6,1.0,2.0,0.05,1.0,100.0,1.0,5.0
7,1.0,2.0,0.50,1.0,10.0,1.0,5.0
8,1.5,0.5,0.50,0.0,10.0,1.0,5.0
9,1.5,2.0,0.50,1.0,10.0,1.0,5.0


In [26]:
column_names = ['alpha','dx','dt','stable','Ninteration',
                'erreur', 'tmax']
#raw_dataset = pd.read_csv('data.dat', names=column_names,
#                      sep=" ",nrows = 10000)

#data = raw_dataset.copy()
data = df
data.columns = ['alpha','dx','dt','stable','Ninteration',
                'erreur', 'tmax']
data.tail()

,alpha,dx,dt,stable,Ninteration,erreur,tmax
5,1.0,0.5,0.50,0.0,10.0,1.0,5.0
6,1.0,2.0,0.05,1.0,100.0,1.0,5.0
7,1.0,2.0,0.50,1.0,10.0,1.0,5.0
8,1.5,0.5,0.50,0.0,10.0,1.0,5.0
9,1.5,2.0,0.50,1.0,10.0,1.0,5.0


# Filtration des donnés

Flitre les donnes que ne sont pas stable
Regroupe les donnes stable en groupes avec le meme pair aplha et dx. Dans chaque groupe il voit la liste des tmax et returne les index avec le plus grand valeur. A la fin, nous avons un liste avec le index de les plus grand valeurs de tmax dans chaque groupe. Aprés ça, il filtre le donnes stable avec seulement les index trouvés.

In [31]:
data_stable = data[data['stable'] == 1]
data_stable.tail()

index_dt_optimal = data_stable.groupby(['alpha','dx'])['Ninteration'].agg(pd.Series.idxmin)
data_stable_optimal = data_stable.filter(items = index_dt_optimal.values, axis = 0)

data_stable_optimal.tail()

,alpha,dx,dt,stable,Ninteration,erreur,tmax
0.0,0.5,0.5,0.05,1.0,100.0,1.0,5.0
3.0,0.5,2.0,0.50,1.0,10.0,1.0,5.0
4.0,1.0,0.5,0.05,1.0,100.0,1.0,5.0
7.0,1.0,2.0,0.50,1.0,10.0,1.0,5.0
9.0,1.5,2.0,0.50,1.0,10.0,1.0,5.0


In [32]:
data_stable_optimal_clean=data_stable_optimal.drop(['stable','Ninteration','erreur', 'tmax'],axis=1)
data_stable_optimal_clean.tail()

,alpha,dx,dt
0.0,0.5,0.5,0.05
3.0,0.5,2.0,0.50
4.0,1.0,0.5,0.05
7.0,1.0,2.0,0.50
9.0,1.5,2.0,0.50


In [33]:
x = data_stable_optimal_clean.values #returns a numpy array
#scaler = preprocessing.MinMaxScaler()
scaler = preprocessing.StandardScaler()
x_scaled = scaler.fit_transform(x)

                          
data_stable_optimal_clean = pd.DataFrame(x_scaled)
data_stable_optimal_clean.columns = ['alpha','dx','dt']
data_stable_optimal_clean.tail()

,alpha,dx,dt
0,-1.069045,-1.224745,-1.224745
1,-1.069045,0.816497,0.816497
2,0.267261,-1.224745,-1.224745
3,0.267261,0.816497,0.816497
4,1.603567,0.816497,0.816497


In [ ]:
train_dataset = data_stable_optimal.sample(frac=0.75,random_state=0)
test_dataset = data_stable_optimal.drop(train_dataset.index)

train_labels = train_dataset.pop('dt')
test_labels = test_dataset.pop('dt')

train_labels = np.asarray(train_labels)
test_labels = np.asarray(test_labels)
train_dataset = np.asarray(train_dataset)
test_dataset = np.asarray(test_dataset)

# Model and network construction

In [ ]:
def build_model():
    from tensorflow import keras
    from tensorflow.keras import layers
    from tensorflow.keras.layers import LeakyReLU

    model = keras.Sequential([
    layers.Dense(10, use_bias=True,activation='relu', input_shape=[train_dataset.shape[1]]),    
    layers.Dense(10, use_bias=True,activation='relu'),     
    layers.Dense(1, use_bias=True,activation='linear'),
    ])

    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
def train_model():
  
    
  optimizer = tf.optimizers.Adam() #

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mse'])

# The patience parameter is the amount of epochs to check for improvement
  early_stop = keras.callbacks.EarlyStopping(monitor='val_mse',min_delta=1e-17, patience=10)
  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1,profile_batch=0)


  model.fit(train_dataset, 
            train_labels, 
            epochs=100,
            batch_size=1000,
            validation_split = 0.5, 
            verbose=0, 
            callbacks=[early_stop,tensorboard_callback])

In [ ]:
#Check that everything is fine
example_batch = train_dataset[:10]
example_result = model.predict(example_batch)
example_result

In [ ]:
# Load the TensorBoard notebook extension to follow ANN training
%reload_ext tensorboard

In [ ]:
#Check results using tensorboard
%tensorboard --logdir logs
#Train model
train_model()

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features).flatten()
    #R2
    R2=metrics.r2_score(test_labels, predictions)
    #Correlation entre predictions et vraies donnees
    Correl=np.corrcoef(test_labels, predictions)[0, 1]
    print('Model Performance:')
    print('Correlation: {:0.4f}.'.format(Correl))
    print('R2: {:0.4f}.'.format(R2))

In [ ]:
def visualize(model, test_features, test_labels):
    X_test=test_labels
    Y_pred=model.predict(test_features).flatten()
    Y_test=test_labels
    plt.scatter(Y_test,Y_pred, color = 'red',marker=".")
    plt.scatter(Y_test,Y_test, color = 'black',marker=".")
    plt.title('ANN Regression Results')
    plt.xlabel('sgs_exact')
    plt.ylabel('sgs_model')
    plt.show()

In [ ]:
evaluate(model, test_dataset, test_labels)
visualize(model, test_dataset, test_labels)

In [1]:
df = pd.DataFrame(np.array(([1, 1, 7, 4], [1, 2, 6, 3], [7, 2, 8, 0],[7, 2, 9, 1])),
                  #index=['mouse', 'rabbit', 'other', 'bla'],
                  columns=['one', 'stable', 'three', 'four'])
g=df.groupby(['one','stable'])
g
#['four'].agg(pd.Series.idxmax)
#h = df.filter(items = g.values, axis = 0)
#df.drop(['stable', 'four'])

NameError: name 'pd' is not defined